<a href="https://colab.research.google.com/github/asattiraju13/wildfire-detection/blob/analysis/Fire_Confidence_Predictor_%2B_Australia_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_path = 'drive/MyDrive/Colab_Notebooks/fire_confidence_predictor.csv'

In [ ]:
df = pd.read_csv(train_path)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.acq_date = pd.to_datetime(df['acq_date'])
df = df.drop('acq_time',axis=1)

In [ ]:
df.drop(['instrument','type','version','scan','track','acq_time'],axis=1,inplace=True)

In [ ]:
df['day'] = df.acq_date.dt.day
df['month'] = df.acq_date.dt.month

df.drop(['acq_date'],axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.instrument.value_counts()

In [ ]:
df['daynight'].replace({'D':0,'N':1},inplace=True)
df['satellite'].replace({'Terra':0,'Aqua':1},inplace=True)

df = df.drop('instrument', axis = 1)

In [ ]:
X = df.drop('confidence',axis=1)
y = df.confidence

In [ ]:
len(df) * 0.7

In [ ]:
X_train = X[:25207]
X_test = X[25207:]

y_train = y[:25207]
y_test = y[25207:]

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
from xgboost import XGBRegressor

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
cat = CatBoostRegressor(loss_function='RMSE',learning_rate=0.10)
xgb = XGBRegressor(n_estimators = 500, learning_rate = 0.05, max_depth = 6)

In [ ]:
cat.fit(X_train, y_train)

In [ ]:
np.sqrt(mean_squared_error(y_test,cat.predict(X_test)))

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
np.sqrt(mean_squared_error(y_test,xgb.predict(X_test)))

In [ ]:
y_pred = cat.predict(X_test)

In [ ]:
lat = pd.Series(X_test.latitude)

In [ ]:
long = pd.Series(X_test.longitude)

In [ ]:
pred = pd.Series(y_pred)

In [ ]:
actual = pd.Series(y_test)

In [ ]:
map_df = pd.concat([lat,long,actual],axis=1)

In [ ]:
map_df = map_df.reset_index().drop('index',axis=1)

In [ ]:
map_df = pd.concat([map_df, pred], axis=1)

In [ ]:
map_df = map_df.rename(columns={0:'pred'})

In [ ]:
print(map_df.head())
print(map_df.shape)

In [ ]:
import plotly.express as px

fig = px.scatter_geo(map_df,lat='latitude',lon='longitude',color='confidence')
fig.update_layout(title = 'Australia Map', title_x=0.5)
fig.show()

In [ ]:
!pip install catboost